# Part 1 of the exercise 

In [104]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [ ]:
# importing raw code data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url_data= requests.get(url)

soup = BeautifulSoup(url_data.text, 'html.parser')
#print(soup.prettify())
table = soup.body.table.tbody

In [58]:
#creating function to get table data

def table_1(i):
    x1 = i.find_all('td')
    y1 = []
    
    for j in x1:
        if j.a:            
            if (j.a.text):
                y1.append(j.a.text)
                continue
        y1.append(j.string.strip())
        
    return y1

def table_2():    
    data = []  
    
    for tr in table.find_all('tr'):
        j = table_1(tr)
        if len(j) != 3:
            continue
        data.append(j)        
    
    return data

In [ ]:
data = table_2()
columns = ['Postcode', 'Borough', 'Neighbourhood']
data

In [60]:
df = pd.DataFrame(data, columns=columns)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [64]:
df1 = df[df.Borough != 'Not assigned']
df1 = df1.sort_values(by=['Postcode','Borough'])
df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union


In [66]:
df1.describe()

,Postcode,Borough,Neighbourhood
count,210,210,210
unique,103,11,208
top,M9V,Etobicoke,Runnymede
freq,8,44,2


In [70]:
#Consolidating the neighbourhoods that share the postcode

df2 = df1.groupby(['Postcode','Borough']).agg(lambda x: ''.join(', ').join(set(x))).reset_index()
df2

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [81]:
df2.shape

(103, 5)

# Part 2 of the exercise

In [83]:
ll = pd.read_csv('https://cocl.us/Geospatial_data')
ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [160]:
df3_tmp = pd.merge(left=df2,right=ll, left_on='Postcode', right_on='Postal Code')
df3 = df3_tmp[['Postcode','Borough','Neighbourhood','Latitude_y','Longitude_y']]
df3.columns = ['Postal Code','Borough','Neighbourhood','Latitude','Longitude']
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [92]:
df3.shape

(103, 5)

# Part 3 of the exercise

## Consider only Downtown 
### Please note that the in the code I put sc or scaborough sometimes in the the variable names, but those were meant to be for Downtown as the data input is still Downtown (tried Scaborough but changed afterwards)

In [161]:

df4 = df3[df3['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
df4.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Let's get the geographical coordinates of Downtown

In [162]:
#import geocoders
from geopy.geocoders import Nominatim

#import folium
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: \ ^C
/ 

In [165]:
address = 'Downtown Toronto, Canada'

geolocator = Nominatim(user_agent="sc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


## Create map of Downtown using latitude and longitude values

In [166]:

map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df4['Latitude'], df4['Longitude'], df4['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

### Somehow images are not loaded on GitHub so please follow this link for the 2 images in this exercise
https://imgur.com/a/ikKkFnY

## Define Foursquare Credentials and Version

In [107]:
CLIENT_ID = '1Y01OHXCZ2EN5ZYSLRLWIF25AFXW13FGAZP1I3TREPT0PMYS' # your Foursquare ID
CLIENT_SECRET = '5JQEYSSEYQVM1SE1G0XUO5JEG4PWCAESZ43RVO2UX3WUQFKM' # your Foursquare Secret
VERSION = '20200120' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1Y01OHXCZ2EN5ZYSLRLWIF25AFXW13FGAZP1I3TREPT0PMYS
CLIENT_SECRET:5JQEYSSEYQVM1SE1G0XUO5JEG4PWCAESZ43RVO2UX3WUQFKM


## Top 100 venues from the region within radius of 500m

In [167]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [168]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

scaborough_venues = getNearbyVenues(names=df4['Neighbourhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
King, Richmond, Adelaide
Harbourfront East, Toronto Islands, Union Station
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Chinatown, Kensington Market, Grange Park
King and Spadina, CN Tower, Island airport, Bathurst Quay, Railway Lands, South Niagara, Harbourfront West
Stn A PO Boxes 25 The Esplanade
Underground city, First Canadian Place
Christie
Queen's Park


In [169]:
print(scaborough_venues.shape)
scaborough_venues.head()

(1312, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown, St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [170]:
print('There are {} uniques categories.'.format(len(scaborough_venues['Venue Category'].unique())))

There are 206 uniques categories.


## Analyse the neighborhoods

In [171]:
# one hot encoding
sc_onehot = pd.get_dummies(scaborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sc_onehot['Neighbourhood'] = scaborough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [sc_onehot.columns[-1]] + list(sc_onehot.columns[:-1])
sc_onehot = sc_onehot[fixed_columns]

sc_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,"Cabbagetown, St. James Town",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [172]:
sc_onehot.shape

(1312, 207)

In [173]:
sc_grouped = sc_onehot.groupby('Neighbourhood').mean().reset_index()
sc_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,0.000000,...,0.00,0.00,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.00,0.012195
3,"Chinatown, Kensington Market, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.045977,0.000000,0.057471,0.011494,0.000000,0.00,0.000000
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,Church and Wellesley,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.011628,0.000000,0.011628,0.00,0.011628
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000
7,Harbourfront,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,"Harbourfront East, Toronto Islands, Union Station",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000
9,"King and Spadina, CN Tower, Island airport, Ba...",0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


## Top venues

In [174]:
num_top_venues = 5

for hood in sc_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = sc_grouped[sc_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2                Café  0.04
3            Beer Bar  0.04
4  Seafood Restaurant  0.04


----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2   Restaurant  0.06
3         Park  0.04
4  Pizza Place  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1  Italian Restaurant  0.05
2                Café  0.04
3        Burger Joint  0.04
4      Ice Cream Shop  0.04


----Chinatown, Kensington Market, Grange Park----
                   venue  freq
0                   Café  0.07
1     Chinese Restaurant  0.06
2  Vietnamese Restaurant  0.06
3    Dumpling Restaurant  0.05
4            Coffee Shop  0.05


----Christie----
           venue  freq
0           Café  0.18
1  Grocery Store  0.18
2           Park  0.12
3     Baby Store  0.06
4          Diner  0.06


----Church and Wellesley----
                 venue  f

In [175]:
#write to pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [176]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = sc_grouped['Neighbourhood']

for ind in np.arange(sc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Steakhouse,Cheese Shop,Café,Italian Restaurant
1,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Park,Pub,Pizza Place,Bakery,Chinese Restaurant,Italian Restaurant,Jewelry Store
2,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Japanese Restaurant,Juice Bar,Sandwich Place,Burger Joint,Café,Spa,Department Store
3,"Chinatown, Kensington Market, Grange Park",Café,Chinese Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Bakery,Bar,Mexican Restaurant,Noodle House
4,Christie,Café,Grocery Store,Park,Gas Station,Italian Restaurant,Baby Store,Restaurant,Athletics & Sports,Diner,Candy Store


## Cluster Neighborhoods

In [177]:
# set number of clusters
kclusters = 5

sc_grouped_clustering = sc_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 3, 0, 0, 0, 0, 4], dtype=int32)

In [178]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sc_merged = df4

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sc_merged = sc_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

sc_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Playground,Trail,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Restaurant,Café,Park,Pub,Pizza Place,Bakery,Chinese Restaurant,Italian Restaurant,Jewelry Store
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Café,Gym,Hotel,Gastropub,Mediterranean Restaurant
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Mexican Restaurant,Shoe Store,Brewery
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Plaza,Restaurant,Pizza Place


In [179]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sc_merged['Latitude'], sc_merged['Longitude'], sc_merged['Neighbourhood'], sc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=9,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


## Examine Cluster

In [180]:
sc_merged.loc[sc_merged['Cluster Labels'] == 0, sc_merged.columns[[1] + list(range(5, sc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Park,Pub,Pizza Place,Bakery,Chinese Restaurant,Italian Restaurant,Jewelry Store
2,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Café,Gym,Hotel,Gastropub,Mediterranean Restaurant
3,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Mexican Restaurant,Shoe Store,Brewery
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Plaza,Restaurant,Pizza Place
5,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Bakery,Cocktail Bar,Hotel,Breakfast Spot,Beer Bar,Cosmetics Shop,Clothing Store
6,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Steakhouse,Cheese Shop,Café,Italian Restaurant
7,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Ice Cream Shop,Japanese Restaurant,Juice Bar,Sandwich Place,Burger Joint,Café,Spa,Department Store
8,Downtown Toronto,0,Coffee Shop,Bar,Café,Steakhouse,Restaurant,Burger Joint,Hotel,Thai Restaurant,Asian Restaurant,Cosmetics Shop
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Italian Restaurant,Brewery,Fried Chicken Joint,Scenic Lookout,Sporting Goods Shop
10,Downtown Toronto,0,Coffee Shop,Café,Hotel,Bar,Restaurant,Gastropub,Seafood Restaurant,Steakhouse,Deli / Bodega,Salad Place


In [181]:
sc_merged.loc[sc_merged['Cluster Labels'] == 1, sc_merged.columns[[1] + list(range(5, sc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Downtown Toronto,1,Coffee Shop,Park,Gym,College Auditorium,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Café,Portuguese Restaurant


In [182]:
sc_merged.loc[sc_merged['Cluster Labels'] == 2, sc_merged.columns[[1] + list(range(5, sc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Park,Playground,Trail,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


In [183]:
sc_merged.loc[sc_merged['Cluster Labels'] == 3, sc_merged.columns[[1] + list(range(5, sc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,3,Café,Grocery Store,Park,Gas Station,Italian Restaurant,Baby Store,Restaurant,Athletics & Sports,Diner,Candy Store


In [184]:
sc_merged.loc[sc_merged['Cluster Labels'] == 4, sc_merged.columns[[1] + list(range(5, sc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,4,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Coffee Shop,Sculpture Garden,Boutique,Rental Car Location,Bar,Plane


# Conclusions

Cluster 0 and 1 are most popular with Coffee Shops but cluster 1 is like central area with restaurants and bars whereas cluster 1 is more for park and gym; 
Cluster 2 is popular with park and playground;
Cluster 3 is mostly Cafe;
Cluster 4 is Airport area. 